In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import os
import time  # Para medir o tempo
import itertools  # Para plotar a matriz de confusão
from tensorflow.keras.applications import (
    DenseNet201,
    ResNet152V2,
    NASNetLarge,
    VGG19,
    Xception,
    InceptionV3,
    InceptionResNetV2,
    MobileNetV2
)
import pandas as pd

In [3]:
# Diretório dos dados
DATA_DIR = '../DATA/classification'

# Parâmetros de treinamento
BATCH_SIZE = 32
IMG_SIZE = (224, 224)  # Será ajustado conforme o modelo
EPOCHS = 30

NUM_CLASSES = 3  # benign, malignant, normal

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    validation_split=0.15  # 15% dos dados para validação
)


# Gerador para o conjunto de treinamento
train_generator = datagen.flow_from_directory(
    directory=DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=['benign', 'malignant', 'normal'],
)

# Gerador para o conjunto de validação
validation_generator = datagen.flow_from_directory(
    directory=DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=['benign', 'malignant', 'normal'],
)

# Obter os nomes das classes a partir do gerador
class_labels = list(train_generator.class_indices.keys())

Found 665 images belonging to 3 classes.
Found 115 images belonging to 3 classes.


In [5]:
def create_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Camada de pooling global
    x = Dense(1024, activation='relu')(x)  # Camada totalmente conectada
    predictions = Dense(num_classes, activation='softmax')(x)  # Camada de saída
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [6]:


models_to_evaluate = [
    ('DenseNet201', DenseNet201),
    ('ResNet152V2', ResNet152V2),
    ('NASNetLarge', NASNetLarge),
    ('VGG19', VGG19),
    ('Xception', Xception),
    ('InceptionV3', InceptionV3),
    ('InceptionResNetV2', InceptionResNetV2),
    ('MobileNetV2', MobileNetV2)
]

In [7]:
results = []

for model_name, model_function in models_to_evaluate:
    print(f"\nTreinando o modelo: {model_name}")
    
    # Ajustar o tamanho da imagem conforme o requisito do modelo
    if model_name == 'NASNetLarge':
        IMG_SIZE = (331, 331)
    elif model_name in ['InceptionV3', 'Xception', 'InceptionResNetV2']:
        IMG_SIZE = (299, 299)
    else:
        IMG_SIZE = (224, 224)
    
    # Atualizar os geradores com o novo tamanho de imagem
    train_generator = datagen.flow_from_directory(
        directory=DATA_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True,
        classes=['benign', 'malignant', 'normal'],
    )

    # Gerador para o conjunto de validação
    validation_generator = datagen.flow_from_directory(
        directory=DATA_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False,
        classes=['benign', 'malignant', 'normal'],
    )
    
    # Obter os nomes das classes
    class_labels = list(train_generator.class_indices.keys())
    
    # Carregar o modelo base pré-treinado
    base_model = model_function(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
    
    # Congelar as camadas do modelo base
    for layer in base_model.layers:
        layer.trainable = False
    
    # Criar o modelo completo
    model = create_model(base_model, num_classes=NUM_CLASSES)
    
    # Compilar o modelo
    learning_rate = 0.001  # Ajuste conforme necessário
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Medir o tempo de treinamento
    start_time = time.time()
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        verbose=True
    )
    training_time = time.time() - start_time
    
    # Medir o tempo de teste e fazer previsões
    start_time = time.time()
    Y_pred = model.predict(validation_generator)
    testing_time = time.time() - start_time
    y_pred = np.argmax(Y_pred, axis=1)
    
    # Obter as classes verdadeiras
    y_true = validation_generator.classes
    
    # Relatório de classificação
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    
    # Matriz de confusão
    cm = confusion_matrix(y_true, y_pred)
    
    # Cálculo das curvas ROC e AUC
    fpr = {}
    tpr = {}
    roc_auc = {}
    for i in range(NUM_CLASSES):
        fpr[i], tpr[i], _ = roc_curve(
            to_categorical(y_true, num_classes=NUM_CLASSES)[:, i],
            Y_pred[:, i]
        )
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Salvar os resultados
    results.append({
        'model_name': model_name,
        'history': history,
        'training_time': training_time,
        'testing_time': testing_time,
        'classification_report': report,
        'confusion_matrix': cm,
        'fpr': fpr,
        'tpr': tpr,
        'roc_auc': roc_auc
    })
    
    # Exibir as métricas
    print(f"Tempo de Treinamento: {training_time:.2f} segundos")
    print(f"Tempo de Teste: {testing_time:.2f} segundos")
    print("Relatório de Classificação:")
    print(classification_report(y_true, y_pred, target_names=class_labels))


Treinando o modelo: DenseNet201
Found 665 images belonging to 3 classes.
Found 115 images belonging to 3 classes.
Epoch 1/30


/Users/matheus/faculdade/TI0160-VisaoComputacionalAplicadaASaude-UFC-2024.1/.env/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step - accuracy: 0.4458 - loss: 1.8944 - val_accuracy: 0.5739 - val_loss: 0.8333
Epoch 2/30
11/21 ━━━━━━━━━━━━━━━━━━━━ 36s 4s/step - accuracy: 0.6917 - loss: 0.7102

In [ ]:
def plot_confusion_matrix(cm, classes,
                          model_name,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    Normalização pode ser aplicada definindo `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de Confusão Normalizada")
    else:
        print('Matriz de Confusão Sem Normalização')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(f'{title} - {model_name}')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Classe Verdadeira')
    plt.xlabel('Classe Predita')
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_roc_curves(fpr, tpr, roc_auc, classes, model_name):
    plt.figure()
    for i in range(len(classes)):
        plt.plot(fpr[i], tpr[i],
                 label='Classe {0} (AUC = {1:0.2f})'
                       ''.format(classes[i], roc_auc[i]))
    plt.plot([0, 1], [0, 1], 'k--')  # Linha diagonal
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taxa de Falsos Positivos')
    plt.ylabel('Taxa de Verdadeiros Positivos')
    plt.title(f'Curvas ROC - {model_name}')
    plt.legend(loc="lower right")
    plt.show()

In [1]:

def plot_training_history(history, model_name):
    """
    Plota a história de treinamento do modelo.
    
    Args:
    history: História de treinamento retornada pelo método fit do Keras.
    model_name: Nome do modelo.
    """
    # Plotar a acurácia de treinamento e validação
    plt.figure(figsize=(12, 4))
    
    # Acurácia
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
    plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
    plt.title(f'Acurácia - {model_name}')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    
    # Perda
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Perda de Treinamento')
    plt.plot(history.history['val_loss'], label='Perda de Validação')
    plt.title(f'Perda - {model_name}')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Criar um DataFrame para coletar as métricas globais
df_results = pd.DataFrame(columns=['Modelo', 'Acurácia Global', 'F1-Score Global', 'Tempo de Treino (s)', 'Tempo de Teste (s)'])

results_list = []

for result in results:
    model_name = result['model_name']
    report = result['classification_report']
    
    # Acurácia global
    accuracy = report['accuracy']
    # F1-Score global (calculado como a média dos F1-Scores das classes)
    f1_score = np.mean([report[label]['f1-score'] for label in class_labels])
    
    # Adicionar ao DataFrame
    results_list.append({
        'Modelo': model_name,
        'Acurácia Global': accuracy,
        'F1-Score Global': f1_score,
        'Tempo de Treino (s)': result['training_time'],
        'Tempo de Teste (s)': result['testing_time']
    })
    
    # Plotar a matriz de confusão
    plot_confusion_matrix(result['confusion_matrix'], classes=class_labels, model_name=model_name, normalize=True)
    
    # Plotar as curvas ROC
    plot_roc_curves(result['fpr'], result['tpr'], result['roc_auc'], class_labels, model_name)

    # Plotar a história de treinamento
    plot_training_history(history, model_name)

# Converter a lista de resultados em um DataFrame
df_results = pd.DataFrame(results_list)

# Exibir o DataFrame com os resultados
print(df_results)
df_results.to_csv('results.csv', index=False)

In [ ]:
for result in results:
    model_name = result['model_name']
    report = result['classification_report']
    print(f"\nMétricas por classe para o modelo: {model_name}")
    for label in class_labels:
        print(f"Classe: {label}")
        print(f" - Precision: {report[label]['precision']:.4f}")
        print(f" - Recall: {report[label]['recall']:.4f}")
        print(f" - F1-score: {report[label]['f1-score']:.4f}")